<a href="https://colab.research.google.com/github/sergekamanzi/ChatBotpro/blob/main/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install --upgrade datasets

In [6]:
import pandas as pd

data = pd.read_csv("/content/Rwanda_History_QA.csv")

data.head()

,Question,Answer
0,What is a key fact about Rwanda's early histor...,A key fact about Rwanda's early history period...
1,What is a key fact about Rwanda's kingdoms per...,A key fact about Rwanda's kingdoms period is t...
2,What is a key fact about Rwanda's colonial rul...,A key fact about Rwanda's colonial rule period...
3,What is a key fact about Rwanda's independence...,A key fact about Rwanda's independence period ...
4,What is a key fact about Rwanda's genocide per...,A key fact about Rwanda's genocide period is t...


In [4]:
data.shape

(700, 2)

In [5]:
data.isna().sum()

,0
Question,0
Answer,0


In [19]:
# Install dependencies
!pip install transformers datasets gradio --quiet

# Imports
import pandas as pd
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments, TextDataset, DataCollatorForLanguageModeling
import gradio as gr
import os

# Load and preprocess data
df = pd.read_csv("/content/Rwanda_History_QA.csv")

# Format Q&A for GPT2 fine-tuning (prompt-response style)
train_data = ""
for i, row in df.iterrows():
    question = row['Question'].strip()
    answer = row['Answer'].strip()
    train_data += f"User: {question}\nBot: {answer}\n\n"

# Save to .txt for training
with open("rwanda_chat_data.txt", "w") as f:
    f.write(train_data)

# Load tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token  # Important for batching
model = GPT2LMHeadModel.from_pretrained("gpt2")

# Dataset preparation
def load_dataset(file_path, tokenizer):
    return TextDataset(
        tokenizer=tokenizer,
        file_path=file_path,
        block_size=128
    )

train_dataset = load_dataset("rwanda_chat_data.txt", tokenizer)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# Training setup
training_args = TrainingArguments(
    output_dir="./gpt2_rwanda_bot",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=2,
    save_steps=200,
    save_total_limit=2,
    prediction_loss_only=True,
    logging_steps=50
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    data_collator=data_collator,
)

# Train the model
trainer.train()
model.save_pretrained("./gpt2_rwanda_bot")
tokenizer.save_pretrained("./gpt2_rwanda_bot")


In [19]:
from transformers import GPT2LMHeadModel

# Load from safetensors and save in .bin format
model = GPT2LMHeadModel.from_pretrained("./gpt2_rwanda_bot", from_safetensors=True)
model.save_pretrained("./gpt2_rwanda_bot", safe_serialization=False)


In [15]:
!ls ./gpt2_rwanda_bot


checkpoint-200	generation_config.json	runs			 vocab.json
checkpoint-342	merges.txt		special_tokens_map.json
config.json	model.safetensors	tokenizer_config.json
